In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#import dataset
from keras.datasets import mnist 
from sklearn.preprocessing import StandardScaler

2022-10-19 07:59:46.415725: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-19 07:59:46.415839: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
from keras.datasets import mnist

#loading the dataset
(train_X, train_y), (test_X, test_y) = mnist.load_data()

#printing the shapes of the vectors 
#print('X_train: ' + str(train_X.shape))
#print('Y_train: ' + str(train_y.shape))
#print('X_test:  '  + str(test_X.shape))
#print('Y_test:  '  + str(test_y.shape))

In [3]:
data = pd.read_csv ('./train.csv')

In [ ]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffl
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape
X_train.shape

(784, 41000)

In [4]:
X_train = train_X.reshape(60000,784).T/255
Y_train = train_y
m = 60000

In [5]:
X_train.shape

(784, 60000)

In [ ]:
plt.figure(figsize= (20,10))
for i in range (9) :
    plt.subplot(330 +1 + i)
    plt.imshow(X_train[:,i].reshape(28,28)*255,cmap= plt.get_cmap('gray'))
    plt.title ("Image label : {}".format(Y_train[i]))

TypeError: 'NoneType' object is not callable

TypeError: 'NoneType' object is not callable

In [52]:
def init_params():
    W1 = np.random.rand(12, 784)
    b1 = np.random.rand(12, 1)
    W2 = np.random.rand(10, 12)
    b2 = np.random.rand(10, 1)
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A


def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y
def batch_normalization (Z , gamma , beta):
    mean = np.mean (Z , axis = 0) # 1 x 
    sigma = (1 /m) * (np.sum (Z-mean)**2)
    Z_norm = (Z-mean) / (np.sqrt(sigma + 0.00001))
    Z = gamma * Z_norm + beta
    return Z
def forward_prop(W1, b1, W2, b2, gamma , beta ,X):
    scaler = StandardScaler()
    Z1 = (W1.dot(X) + b1)
    Z1 = scaler.fit_transform (Z1)
    A1 = ReLU(Z1)
    Z2 = (W2.dot(A1) + b2)
    Z2 = scaler.fit_transform (Z2)
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [61]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations,W1, b1, W2, b2):
    beta = np.random.randn()
    gamma = np.random.randn()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, gamma , beta ,X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2


In [62]:
W1, b1, W2, b2 = init_params()
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.05, 1000 , W1, b1, W2, b2)

Iteration:  0
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7698333333333334
Iteration:  10
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7697833333333334
Iteration:  20
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7698
Iteration:  30
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7699
Iteration:  40
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7699
Iteration:  50
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7699666666666667
Iteration:  60
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7700166666666667
Iteration:  70
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7700166666666667
Iteration:  80
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.76995
Iteration:  90
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7699
Iteration:  100
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7698
Iteration:  110
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7698166666666667
Iteration:  120
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7697333333333334
Iteration:  130
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7696666666666667
Iteration:  140
[3 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7695166666666666
Iteration:  150
[